In [3]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
import copy

In [1]:
CUR_DIR = os.curdir
PATH_FILE = CUR_DIR + '/fileCSVNow/fileCSV - Copy/'

In [4]:
coin_list = os.listdir(PATH_FILE)
dict_checkbox = {}
sel_all = widgets.Checkbox(
    value=False,
    description='all',
    disabled=False,
    indent=False
)
for i in range(len(coin_list)):
    file_name = coin_list[i].split('-')
    coin_name = file_name[0]
    for j in file_name[1:-1]:
        # print(i)
        if j != 'usd':
            coin_name = coin_name + '-' + j
        else:
            break
    coin_list[i] = coin_name
    dict_checkbox[coin_name] = widgets.Checkbox(
                                value=False,
                                description=coin_name,
                                disabled=False,
                                indent=False,
                                continous_update=True
                            )
    widgets.dlink((sel_all, 'value'), (dict_checkbox[coin_name], 'value'))


DISP_COL = 20

dict_vbox = {}
for i in range(DISP_COL):
    dict_vbox[i] = []
dict_vbox[0].append(sel_all)

for i in range(1,len(coin_list)+1):
    coord = i%DISP_COL
    dict_vbox[coord].append(dict_checkbox[coin_list[i-1]])

hbox = [widgets.VBox(i) for i in dict_vbox.values()]
display(widgets.HBox(hbox))

In [5]:
list_checked = []
for i in coin_list:
    if dict_checkbox[i].value:
        list_checked.append(i)

df_checked = {}
for i in list_checked:
    df_checked[i] = pd.read_csv(PATH_FILE + i + '-usd-max.csv')
    df_checked[i]['pct_change'] = df_checked[i]['price'].pct_change()
    df_checked[i]['pct_change'] = np.where(df_checked[i]['pct_change'] > 10**3, 10, df_checked[i]['pct_change'])


In [52]:
PUMP_THRESHOLD = 1
num_pump = 0
for i in list_checked:
    num_pump = num_pump + len(df_checked[i][df_checked[i]['pct_change'] > PUMP_THRESHOLD])

# is_pump on a single day

In [6]:
PUMP_THRESHOLD = 1
WINDOW_BEFORE = 15
WINDOW_AFTER = 7

data_x = []
data_y = []
for i in tqdm(list_checked):
    max_idx = max(df_checked[i].index)
    df_checked_dropna = df_checked[i].dropna()
    for j in df_checked_dropna.index:
        if j + WINDOW_BEFORE < max_idx:
            df_temp = df_checked_dropna.iloc[j:j+WINDOW_BEFORE]
            one_data = [i, df_temp['snapped_at'].values[0]] + df_temp['pct_change'].values.tolist()
            data_x.append(one_data)
            low_bound = j+WINDOW_BEFORE
            up_bound = min([low_bound+WINDOW_AFTER,max_idx])
            df_temp = df_checked_dropna.iloc[low_bound:up_bound]
            if (df_temp['pct_change'] >= PUMP_THRESHOLD).any():
                data_y.append(1)
            else:
                data_y.append(0)

100%|██████████| 831/831 [07:15<00:00,  1.91it/s]


In [14]:
TEST_SIZE = 0.2

prediktor_real = pd.DataFrame(data_x)
prediktor_real['target'] = data_y
prediktor_real = prediktor_real.dropna()

prediktor = pd.DataFrame(data_x)
prediktor['target'] = data_y
prediktor.replace([np.inf, -np.inf], np.nan, inplace=True)
prediktor = prediktor.dropna()
prediktor_0 = prediktor[prediktor['target'] == 0]
prediktor_1 = prediktor[prediktor['target'] == 1]
prediktor_0 = prediktor_0.sample(n=int(len(prediktor_0)))

prediktor = pd.concat([prediktor_0, prediktor_1]).drop([0,1],1)
target = prediktor['target']
prediktor = prediktor.drop('target',1)


X_train, X_test, y_train, y_test = train_test_split(prediktor, target, test_size= TEST_SIZE)

scale = SMOTE()
X_train,y_train = scale.fit_resample(X_train, y_train)

X = prediktor_real[np.arange(2,17).tolist()]
y = prediktor_real['target']

In [8]:
arr_pred = []
for i in df_checked.keys():
    data_now = df_checked[i].iloc[-WINDOW_BEFORE:]
    arr_pred.append(data_now['pct_change'].values.tolist())
test_now = pd.DataFrame(arr_pred).dropna()

In [13]:
from sklearn.gaussian_process import GaussianProcessClassifier

model = GaussianProcessClassifier().fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_pred = model.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gpr_pred = model.predict(test_now)
gpr_pred_proba = model.predict_proba(test_now)
gpr_pred_coin = {}
for i in range(len(gpr_pred)):
    if gpr_pred[i] == 1:
        gpr_pred_coin[coin_list[i]] = gpr_pred_proba[i][1]
print(gpr_pred_coin)
print(len(gpr_pred_coin))

              precision    recall  f1-score   support

           0       0.69      0.92      0.79      9781
           1       0.88      0.60      0.71      9799

    accuracy                           0.76     19580
   macro avg       0.78      0.76      0.75     19580
weighted avg       0.78      0.76      0.75     19580

              precision    recall  f1-score   support

           0       0.70      0.91      0.79      2457
           1       0.87      0.60      0.71      2439

    accuracy                           0.76      4896
   macro avg       0.79      0.76      0.75      4896
weighted avg       0.78      0.76      0.75      4896

              precision    recall  f1-score   support

           0       0.69      0.91      0.79     12238
           1       0.88      0.60      0.71     12238

    accuracy                           0.76     24476
   macro avg       0.78      0.76      0.75     24476
weighted avg       0.78      0.76      0.75     24476

False Positive Perc

In [65]:
from sklearn.gaussian_process import GaussianProcessClassifier

model = GaussianProcessClassifier().fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_pred = model.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))


              precision    recall  f1-score   support

           0       0.70      0.92      0.79      9768
           1       0.88      0.60      0.72      9796

    accuracy                           0.76     19564
   macro avg       0.79      0.76      0.75     19564
weighted avg       0.79      0.76      0.75     19564

              precision    recall  f1-score   support

           0       0.70      0.92      0.79      2460
           1       0.88      0.59      0.71      2432

    accuracy                           0.76      4892
   macro avg       0.79      0.76      0.75      4892
weighted avg       0.79      0.76      0.75      4892

              precision    recall  f1-score   support

           0       0.70      0.92      0.79     12228
           1       0.88      0.60      0.71     12228

    accuracy                           0.76     24456
   macro avg       0.79      0.76      0.75     24456
weighted avg       0.79      0.76      0.75     24456



In [15]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier().fit(X_train, y_train)

y_train_pred = model1.predict(X_train)
y_test_pred = model1.predict(X_test)
y_pred = model1.predict(X.dropna())

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

knn_pred = model1.predict(test_now)
knn_pred_proba = model1.predict_proba(test_now)
knn_pred_coin = {}
for i in range(len(knn_pred)):
    if knn_pred[i] == 1:
        knn_pred_coin[coin_list[i]] = knn_pred_proba[i][1]
print(knn_pred_coin)
print(len(knn_pred_coin))



              precision    recall  f1-score   support

           0       1.00      0.95      0.98    555453
           1       0.95      1.00      0.98    555453

    accuracy                           0.98   1110906
   macro avg       0.98      0.98      0.98   1110906
weighted avg       0.98      0.98      0.98   1110906

              precision    recall  f1-score   support

           0       0.99      0.93      0.96    138842
           1       0.05      0.39      0.09      1241

    accuracy                           0.93    140083
   macro avg       0.52      0.66      0.53    140083
weighted avg       0.99      0.93      0.96    140083

              precision    recall  f1-score   support

           0       1.00      0.95      0.97    694295
           1       0.13      0.88      0.23      6119

    accuracy                           0.95    700414
   macro avg       0.56      0.91      0.60    700414
weighted avg       0.99      0.95      0.97    700414

False Positive Perc

In [16]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

model2 = AdaBoostClassifier().fit(X_train, y_train)

y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)
y_pred = model2.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

ab_pred = model2.predict(test_now)
ab_pred_proba = model2.predict_proba(test_now)
ab_pred_coin = {}
for i in range(len(ab_pred)):
    if ab_pred[i] == 1:
        ab_pred_coin[coin_list[i]] = ab_pred_proba[i][1]
print(ab_pred_coin)
print(len(ab_pred_coin))


              precision    recall  f1-score   support

           0       0.68      0.88      0.77    555453
           1       0.84      0.59      0.69    555453

    accuracy                           0.74   1110906
   macro avg       0.76      0.74      0.73   1110906
weighted avg       0.76      0.74      0.73   1110906

              precision    recall  f1-score   support

           0       1.00      0.88      0.94    138842
           1       0.05      0.65      0.09      1241

    accuracy                           0.88    140083
   macro avg       0.52      0.77      0.51    140083
weighted avg       0.99      0.88      0.93    140083

              precision    recall  f1-score   support

           0       1.00      0.88      0.94    694295
           1       0.05      0.65      0.09      6119

    accuracy                           0.88    700414
   macro avg       0.52      0.77      0.51    700414
weighted avg       0.99      0.88      0.93    700414

False Positive Perc

In [27]:
from sklearn.tree import DecisionTreeClassifier

model3 = DecisionTreeClassifier().fit(X_train, y_train)

y_train_pred = model3.predict(X_train)
y_test_pred = model3.predict(X_test)
y_pred = model3.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9768
           1       1.00      1.00      1.00      9796

    accuracy                           1.00     19564
   macro avg       1.00      1.00      1.00     19564
weighted avg       1.00      1.00      1.00     19564

              precision    recall  f1-score   support

           0       0.74      0.72      0.73      2460
           1       0.73      0.75      0.74      2432

    accuracy                           0.73      4892
   macro avg       0.73      0.73      0.73      4892
weighted avg       0.73      0.73      0.73      4892

              precision    recall  f1-score   support

           0       0.95      0.94      0.95     12228
           1       0.94      0.95      0.95     12228

    accuracy                           0.95     24456
   macro avg       0.95      0.95      0.95     24456
weighted avg       0.95      0.95      0.95     24456



In [17]:
model4 = RandomForestClassifier().fit(X_train, y_train)

y_train_pred = model4.predict(X_train)
y_test_pred = model4.predict(X_test)
y_pred = model4.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

rf_pred = model4.predict(test_now)
rf_pred_proba = model4.predict_proba(test_now)
rf_pred_coin = {}
for i in range(len(rf_pred)):
    if rf_pred[i] == 1:
        rf_pred_coin[coin_list[i]] = rf_pred_proba[i][1]
print(rf_pred_coin)
print(len(rf_pred_coin))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    555453
           1       1.00      1.00      1.00    555453

    accuracy                           1.00   1110906
   macro avg       1.00      1.00      1.00   1110906
weighted avg       1.00      1.00      1.00   1110906

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    138842
           1       0.20      0.38      0.26      1241

    accuracy                           0.98    140083
   macro avg       0.60      0.68      0.63    140083
weighted avg       0.99      0.98      0.98    140083

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    694295
           1       0.68      0.87      0.76      6119

    accuracy                           1.00    700414
   macro avg       0.84      0.94      0.88    700414
weighted avg       1.00      1.00      1.00    700414

False Positive Perc

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

model5 = GradientBoostingClassifier().fit(X_train, y_train)

y_train_pred = model5.predict(X_train)
y_test_pred = model5.predict(X_test)
y_pred = model5.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gb_pred = model5.predict(test_now)
gb_pred_proba = model5.predict_proba(test_now)
gb_pred_coin = {}
for i in range(len(gb_pred)):
    if gb_pred[i] == 1:
        gb_pred_coin[coin_list[i]] = gb_pred_proba[i][1]
print(gb_pred_coin)
print(len(gb_pred_coin))


              precision    recall  f1-score   support

           0       0.69      0.90      0.78    555453
           1       0.86      0.60      0.70    555453

    accuracy                           0.75   1110906
   macro avg       0.77      0.75      0.74   1110906
weighted avg       0.77      0.75      0.74   1110906

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    138842
           1       0.05      0.65      0.10      1241

    accuracy                           0.90    140083
   macro avg       0.53      0.78      0.52    140083
weighted avg       0.99      0.90      0.94    140083

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    694295
           1       0.05      0.65      0.10      6119

    accuracy                           0.90    700414
   macro avg       0.53      0.78      0.52    700414
weighted avg       0.99      0.90      0.94    700414

False Positive Perc

# Is_pump not in a single day

In [6]:
PUMP_THRESHOLD = 3
WINDOW_BEFORE = 30
WINDOW_AFTER = 30

data_x = []
data_y = []
for i in tqdm(list_checked):
    max_idx = max(df_checked[i].index)
    df_checked_dropna = df_checked[i].dropna()
    for j in df_checked_dropna.index:
        if j + WINDOW_BEFORE < max_idx-1:
            df_temp = df_checked_dropna.iloc[j:j+WINDOW_BEFORE]
            one_data = [i, df_temp['snapped_at'].values[0]] + df_temp['pct_change'].values.tolist()
            try:
                low_bound = j+WINDOW_BEFORE
                up_bound = min([low_bound+WINDOW_AFTER,max_idx+1])
                df_temp_next = df_checked_dropna.iloc[low_bound:up_bound]
                max_price = max(df_temp_next['price'])
                pct_to_max = (max_price - df_temp['price'].values[-1])/df_temp['price'].values[-1]
                if  pct_to_max > PUMP_THRESHOLD:
                    data_y.append(1)
                else:
                    data_y.append(0)
                data_x.append(one_data)
            except: continue

100%|██████████| 831/831 [05:55<00:00,  2.34it/s]


In [11]:
TEST_SIZE = 0.2

prediktor_real = pd.DataFrame(data_x)
prediktor_real['target'] = data_y

prediktor = pd.DataFrame(data_x)
prediktor['target'] = data_y
prediktor.replace([np.inf, -np.inf], np.nan, inplace=True)
prediktor = prediktor.dropna()
prediktor_0 = prediktor[prediktor['target'] == 0]
prediktor_1 = prediktor[prediktor['target'] == 1]
prediktor_0 = prediktor_0.sample(n=int(len(prediktor_0)))

prediktor = pd.concat([prediktor_0, prediktor_1]).drop([0,1],1)
target = prediktor['target']
prediktor = prediktor.drop('target',1)


X_train, X_test, y_train, y_test = train_test_split(prediktor, target, test_size= TEST_SIZE)

scale = SMOTE()
X_train,y_train = scale.fit_resample(X_train, y_train)

X = prediktor_real[np.arange(2,2+WINDOW_AFTER).tolist()]
y = prediktor_real['target']

In [8]:
arr_pred = []
for i in df_checked.keys():
    data_now = df_checked[i].iloc[-WINDOW_BEFORE:]
    arr_pred.append(data_now['pct_change'].values.tolist())
test_now = pd.DataFrame(arr_pred).dropna()


In [40]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier().fit(X_train, y_train)

y_train_pred = model1.predict(X_train)
y_test_pred = model1.predict(X_test)
y_pred = model1.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

knn_pred = model1.predict(test_now)
knn_pred_proba = model1.predict_proba(test_now)
knn_pred_coin = {}
for i in range(len(knn_pred)):
    if knn_pred[i] == 1:
        knn_pred_coin[coin_list[i]] = knn_pred_proba[i][1]
print(knn_pred_coin)
print(len(knn_pred_coin))



              precision    recall  f1-score   support

           0       0.74      0.85      0.80      9690
           1       0.83      0.71      0.76      9690

    accuracy                           0.78     19380
   macro avg       0.79      0.78      0.78     19380
weighted avg       0.79      0.78      0.78     19380

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      2388
           1       0.74      0.56      0.64      2444

    accuracy                           0.68      4832
   macro avg       0.69      0.68      0.68      4832
weighted avg       0.69      0.68      0.68      4832

              precision    recall  f1-score   support

           0       0.99      0.79      0.88    687506
           1       0.05      0.68      0.10     12078

    accuracy                           0.78    699584
   macro avg       0.52      0.73      0.49    699584
weighted avg       0.98      0.78      0.86    699584

False Positive Perc

In [13]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

model2 = AdaBoostClassifier().fit(X_train, y_train)

y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)
y_pred = model2.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

ab_pred = model2.predict(test_now)
ab_pred_proba = model2.predict_proba(test_now)
ab_pred_coin = {}
for i in range(len(ab_pred)):
    if ab_pred[i] == 1:
        ab_pred_coin[coin_list[i]] = ab_pred_proba[i][1]
print(ab_pred_coin)
print(len(ab_pred_coin))


              precision    recall  f1-score   support

           0       0.61      0.71      0.66    536920
           1       0.65      0.55      0.60    536920

    accuracy                           0.63   1073840
   macro avg       0.63      0.63      0.63   1073840
weighted avg       0.63      0.63      0.63   1073840

              precision    recall  f1-score   support

           0       0.98      0.70      0.82    134249
           1       0.04      0.52      0.07      3191

    accuracy                           0.70    137440
   macro avg       0.51      0.61      0.45    137440
weighted avg       0.96      0.70      0.80    137440

              precision    recall  f1-score   support

           0       0.98      0.71      0.82    671169
           1       0.04      0.55      0.08     16029

    accuracy                           0.70    687198
   macro avg       0.51      0.63      0.45    687198
weighted avg       0.96      0.70      0.81    687198

False Positive Perc

In [14]:
model4 = RandomForestClassifier().fit(X_train, y_train)

y_train_pred = model4.predict(X_train)
y_test_pred = model4.predict(X_test)
y_pred = model4.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

rf_pred = model4.predict(test_now)
rf_pred_proba = model4.predict_proba(test_now)
rf_pred_coin = {}
for i in range(len(rf_pred)):
    if rf_pred[i] == 1:
        rf_pred_coin[coin_list[i]] = rf_pred_proba[i][1]
print(rf_pred_coin)
print(len(rf_pred_coin))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    536920
           1       1.00      1.00      1.00    536920

    accuracy                           1.00   1073840
   macro avg       1.00      1.00      1.00   1073840
weighted avg       1.00      1.00      1.00   1073840

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    134249
           1       0.19      0.07      0.10      3191

    accuracy                           0.97    137440
   macro avg       0.59      0.53      0.54    137440
weighted avg       0.96      0.97      0.97    137440

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    671169
           1       0.91      0.81      0.86     16029

    accuracy                           0.99    687198
   macro avg       0.95      0.91      0.93    687198
weighted avg       0.99      0.99      0.99    687198

False Positive Perc

In [47]:
from sklearn.ensemble import GradientBoostingClassifier

model5 = GradientBoostingClassifier().fit(X_train, y_train)

y_train_pred = model5.predict(X_train)
y_test_pred = model5.predict(X_test)
y_pred = model5.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gb_pred = model5.predict(test_now)
gb_pred_proba = model5.predict_proba(test_now)
gb_pred_coin = {}
for i in range(len(gb_pred)):
    if gb_pred[i] == 1:
        gb_pred_coin[coin_list[i]] = gb_pred_proba[i][1]
print(gb_pred_coin)
print(len(gb_pred_coin))


              precision    recall  f1-score   support

           0       0.66      0.79      0.72    550036
           1       0.74      0.60      0.66    550036

    accuracy                           0.69   1100072
   macro avg       0.70      0.69      0.69   1100072
weighted avg       0.70      0.69      0.69   1100072

              precision    recall  f1-score   support

           0       0.99      0.79      0.88    137470
           1       0.05      0.63      0.09      2447

    accuracy                           0.78    139917
   macro avg       0.52      0.71      0.48    139917
weighted avg       0.98      0.78      0.86    139917

              precision    recall  f1-score   support

           0       0.99      0.79      0.88    687506
           1       0.05      0.65      0.09     12078

    accuracy                           0.78    699584
   macro avg       0.52      0.72      0.49    699584
weighted avg       0.98      0.78      0.86    699584

False Positive Perc

In [34]:
from sklearn.gaussian_process import GaussianProcessClassifier

model = GaussianProcessClassifier().fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
n_loop = int(len(prediktor_real)/len(X_train))
y_pred = []
for i in range(n_loop+1):
    print((i+1)/n_loop)
    right_bound = np.min([(i+1)*len(X_train),len(prediktor_real)])
    X_partial = prediktor_real.iloc[i*len(X_train):right_bound]
    y_target = X_partial['target']
    X_partial = X_partial[np.arange(2,17).tolist()]
    y_pred = y_pred + model.predict(X_partial).tolist()
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
print(classification_report(prediktor_real['target'], y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gpr_pred = model.predict(test_now)
gpr_pred_proba = model.predict_proba(test_now)
gpr_pred_coin = {}
for i in range(len(gpr_pred)):
    if gpr_pred[i] == 1:
        gpr_pred_coin[coin_list[i]] = gpr_pred_proba[i][1]
print(gpr_pred_coin)
print(len(gpr_pred_coin))

              precision    recall  f1-score   support

           0       0.99      0.84      0.91    687506
           1       0.06      0.58      0.11     12078

    accuracy                           0.84    699584
   macro avg       0.53      0.71      0.51    699584
weighted avg       0.98      0.84      0.90    699584

False Positive Percentage:  0.2058660763696735  FP:  372  TP:  1435
False Negative Rate:  0.331900826446281  FN:  1004  TN:  2021
{'agld': 0.6149510519261696, 'alcx': 0.5339189119367447, 'anc': 0.6658286331525574, 'aog': 0.5000212312899125, 'arpa': 0.5116871089946677, 'ast': 0.6657068442368086, 'bor': 0.7848289992141417, 'boring': 0.5681592561272311, 'box': 0.804528048971406, 'bwo': 0.813820245805573, 'cdt': 0.5947313592001251, 'ceek': 0.7381073308721966, 'ctxc': 0.6410255000731411, 'cvxcrv': 0.624233222060866, 'deto': 0.5381761337185367, 'dext': 0.5273484042645364, 'dvf': 0.5072101474961777, 'elg': 0.5439579500205127, 'firo': 0.8511377685862271, 'fun': 0.632708884

In [25]:
DATE_LIST = np.unique(prediktor_real[1])
DATE_LAST = 365

data_pump = prediktor_real[prediktor_real['target'] == 1]
data_pump['target_pred'] = y_pred

dict_num_pump = {}
dict_num_pump_pred = {}
for i in DATE_LIST[-DATE_LAST:]:
    temp_df = data_pump[data_pump[1] == i]
    dict_num_pump[i] = len(temp_df)
    dict_num_pump_pred[i] = len(temp_df[temp_df['target_pred'] == 1])

c:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
import plotly.graph_objects as go

fig = go.Figure([go.Bar(x=list(dict_num_pump.keys()), y=list(dict_num_pump.values()))])
fig.show()

fig = go.Figure([go.Bar(x=list(dict_num_pump_pred.keys()), y=list(dict_num_pump_pred.values()))])
fig.show()

In [14]:
DATE_SHOW = '2022-06-15 00:00:00 UTC'
df_show = prediktor_real[prediktor_real[1] == DATE_SHOW]
df_show[df_show['target'] == 1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,target
397335,nmr,2022-06-15 00:00:00 UTC,0.009953,0.026725,-0.061326,0.013263,-0.031594,0.030202,0.066239,-0.025231,-0.04168,0.130091,0.015258,0.029318,-0.059703,-0.00321,0.047474,1


# USING MA

In [33]:
ma_used = [7, 20, 50]
df_checked_ma = copy.copy(df_checked)
for i in df_checked_ma.keys():
  for j in ma_used:
    df_checked_ma[i]['ma_' + str(j)] = df_checked_ma[i]['price'].rolling(window=j).mean()

for i in df_checked_ma.keys():
  df_checked_ma[i] = df_checked_ma[i].dropna()

for i in df_checked_ma.keys():
  for j in ma_used:
    df_checked_ma[i]['pct_ma_' + str(j)] = (df_checked_ma[i]['price'] - df_checked_ma[i]['ma_' + str(j)])/df_checked_ma[i]['ma_' + str(j)]

c:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [68]:
PUMP_THRESHOLD = 1
WINDOW_BEFORE = 15
WINDOW_AFTER = 7

data_x = []
data_y = []
for i in tqdm(list_checked):
    try:
        max_idx = max(df_checked_ma[i].index)
        df_checked_dropna = df_checked_ma[i].dropna()
        for j in df_checked_dropna.index:
            if j + WINDOW_BEFORE < max_idx-1:
                df_temp = df_checked_dropna.loc[j:j+WINDOW_BEFORE-1]
                one_data = [i, df_temp['snapped_at'].values[0]] + df_temp['pct_change'].values.tolist() + [df_temp['pct_ma_' + str(k)].values[-1] for k in ma_used]
                try:
                    low_bound = j+WINDOW_BEFORE
                    up_bound = min([low_bound+WINDOW_AFTER,max_idx+1])
                    df_temp_next = df_checked_dropna.loc[low_bound:up_bound]
                    max_price = max(df_temp_next['price'])
                    pct_to_max = (max_price - df_temp['price'].values[-1])/df_temp['price'].values[-1]
                    if  pct_to_max > PUMP_THRESHOLD:
                        data_y.append(1)
                    else:
                        data_y.append(0)
                    data_x.append(one_data)
                except: continue
    except: continue

c:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
100%|██████████| 831/831 [06:40<00:00,  2.07it/s]


In [97]:
TEST_SIZE = 0.2

prediktor_real = pd.DataFrame(data_x)
prediktor_real['target'] = data_y
prediktor_real = prediktor_real.dropna()

prediktor = prediktor_real.drop([0,1],1)
target = prediktor['target']
prediktor = prediktor.drop('target',1)

X_train, X_test, y_train, y_test = train_test_split(prediktor, target, test_size= TEST_SIZE)

prediktor = pd.DataFrame(X_test)
prediktor['target'] = y_test
prediktor.replace([np.inf, -np.inf], np.nan, inplace=True)
prediktor = prediktor.dropna()
prediktor_0 = prediktor[prediktor['target'] == 0]
prediktor_1 = prediktor[prediktor['target'] == 1]

prediktor = pd.concat([prediktor_0, prediktor_1])
y_test = prediktor['target']
X_test = prediktor.drop('target',1)


prediktor = pd.DataFrame(X_train)
prediktor['target'] = y_train
prediktor.replace([np.inf, -np.inf], np.nan, inplace=True)
prediktor = prediktor.dropna()
prediktor_0 = prediktor[prediktor['target'] == 0]
prediktor_1 = prediktor[prediktor['target'] == 1]
prediktor_0 = prediktor_0.sample(n=int(len(prediktor_0)))

prediktor = pd.concat([prediktor_0, prediktor_1])
y_train = prediktor['target']
X_train = prediktor.drop('target',1)

scale = SMOTE()
X_train,y_train = scale.fit_resample(X_train, y_train)

X = prediktor_real.drop([0,1],1)
y = prediktor_real['target']

In [82]:
arr_pred = []
for i in df_checked_ma.keys():
    try:
        data_now = df_checked_ma[i].iloc[-WINDOW_BEFORE:]
        arr_pred.append(data_now['pct_change'].values.tolist() + [data_now['pct_ma_' + str(k)].values[-1] for k in ma_used])
    except: continue
test_now = pd.DataFrame(arr_pred).dropna()


In [99]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

model2 = AdaBoostClassifier().fit(X_train, y_train)

y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)
# y_pred = model2.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

ab_pred = model2.predict(test_now)
ab_pred_proba = model2.predict_proba(test_now)
ab_pred_coin = {}
for i in range(len(ab_pred)):
    if ab_pred[i] == 1:
        ab_pred_coin[coin_list[i]] = ab_pred_proba[i][1]
print(ab_pred_coin)
print(len(ab_pred_coin))


              precision    recall  f1-score   support

           0       0.67      0.77      0.72    518019
           1       0.73      0.63      0.68    518019

    accuracy                           0.70   1036038
   macro avg       0.70      0.70      0.70   1036038
weighted avg       0.70      0.70      0.70   1036038

              precision    recall  f1-score   support

           0       0.99      0.77      0.87    129504
           1       0.05      0.62      0.09      2478

    accuracy                           0.77    131982
   macro avg       0.52      0.70      0.48    131982
weighted avg       0.97      0.77      0.85    131982

False Positive Percentage:  0.9500909208988181  FP:  29259  TP:  1537
False Negative Rate:  0.009299705492854743  FN:  941  TN:  100245
{'adco': 0.5011224427920151, 'anc': 0.5019415819750008, 'aurora': 0.5015291023874403, 'babydoge': 0.5003190070520674, 'bcn': 0.5001724167325184, 'bnt': 0.5005791000119841, 'bor': 0.5004422657966131, 'boring': 0

In [100]:
model4 = RandomForestClassifier().fit(X_train, y_train)

y_train_pred = model4.predict(X_train)
y_test_pred = model4.predict(X_test)
# y_pred = model4.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

rf_pred = model4.predict(test_now)
rf_pred_proba = model4.predict_proba(test_now)
rf_pred_coin = {}
for i in range(len(rf_pred)):
    if rf_pred[i] == 1:
        rf_pred_coin[coin_list[i]] = rf_pred_proba[i][1]
print(rf_pred_coin)
print(len(rf_pred_coin))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    518019
           1       1.00      1.00      1.00    518019

    accuracy                           1.00   1036038
   macro avg       1.00      1.00      1.00   1036038
weighted avg       1.00      1.00      1.00   1036038

              precision    recall  f1-score   support

           0       0.98      0.98      0.98    129504
           1       0.18      0.21      0.20      2478

    accuracy                           0.97    131982
   macro avg       0.58      0.60      0.59    131982
weighted avg       0.97      0.97      0.97    131982

False Positive Percentage:  0.8181506849315069  FP:  2389  TP:  531
False Negative Rate:  0.015085772729385877  FN:  1947  TN:  127115
{'bux': 0.91, 'fida': 0.58, 'fidu': 0.56, 'mxc': 0.67, 'rdpx': 0.68, 'rei': 0.62, 'wnxm': 0.6}
7


In [102]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [200, 500],
    'max_depth' : [4, 8, 12],
    # 'criterion' :['gini', 'entropy']
}

model4 = RandomForestClassifier()

cv_model4 = GridSearchCV(estimator=model4, param_grid=param_grid, cv= 5)
cv_model4.fit(X_train, y_train)
print(cv_model4.best_params_)

{'max_depth': 12, 'n_estimators': 200}


In [103]:
cv_model4.cv_results_

{'mean_fit_time': array([ 394.67731905,  963.68347168,  745.04512391, 1771.3310657 ,
         945.37780786, 2473.71791735]),
 'std_fit_time': array([  5.13253717,  44.58993148,   5.31360743,  48.11548609,
          9.66296047, 149.82966995]),
 'mean_score_time': array([ 2.52139006,  6.06971827,  3.4976284 ,  8.26511464,  4.4900033 ,
        12.15226002]),
 'std_score_time': array([0.27177996, 0.265858  , 0.2835551 , 0.71665571, 0.09729807,
        0.57500583]),
 'param_max_depth': masked_array(data=[4, 4, 8, 8, 12, 12],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[200, 500, 200, 500, 200, 500],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 4, 'n_estimators': 200},
  {'max_depth': 4, 'n_estimators': 500},
  {'max_depth': 8, 'n_estimators': 200},
  {'max_depth': 8, 'n_estimators': 500

In [104]:
model4 = cv_model4
# model4 = RandomForestClassifier().fit(X_train, y_train)

y_train_pred = model4.predict(X_train)
y_test_pred = model4.predict(X_test)
# y_pred = model4.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

rf_pred = model4.predict(test_now)
rf_pred_proba = model4.predict_proba(test_now)
rf_pred_coin = {}
for i in range(len(rf_pred)):
    if rf_pred[i] == 1:
        rf_pred_coin[coin_list[i]] = rf_pred_proba[i][1]
print(rf_pred_coin)
print(len(rf_pred_coin))


              precision    recall  f1-score   support

           0       0.71      0.83      0.76    518019
           1       0.79      0.66      0.72    518019

    accuracy                           0.74   1036038
   macro avg       0.75      0.74      0.74   1036038
weighted avg       0.75      0.74      0.74   1036038

              precision    recall  f1-score   support

           0       0.99      0.82      0.90    129504
           1       0.06      0.57      0.10      2478

    accuracy                           0.81    131982
   macro avg       0.52      0.69      0.50    131982
weighted avg       0.97      0.81      0.88    131982

False Positive Percentage:  0.9438373163604339  FP:  23578  TP:  1403
False Negative Rate:  0.01004663507817684  FN:  1075  TN:  105926
{'10set': 0.5151279104274523, 'adco': 0.5019814457975728, 'anc': 0.520420362725112, 'arpa': 0.6001704218195122, 'asd': 0.500569640754966, 'ast': 0.5181987331723062, 'babydoge': 0.5377118075190446, 'box': 0.5339

In [101]:
from sklearn.ensemble import GradientBoostingClassifier

model5 = GradientBoostingClassifier().fit(X_train, y_train)

y_train_pred = model5.predict(X_train)
y_test_pred = model5.predict(X_test)
# y_pred = model5.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gb_pred = model5.predict(test_now)
gb_pred_proba = model5.predict_proba(test_now)
gb_pred_coin = {}
for i in range(len(gb_pred)):
    if gb_pred[i] == 1:
        gb_pred_coin[coin_list[i]] = gb_pred_proba[i][1]
print(gb_pred_coin)
print(len(gb_pred_coin))


              precision    recall  f1-score   support

           0       0.68      0.77      0.73    518019
           1       0.74      0.64      0.69    518019

    accuracy                           0.71   1036038
   macro avg       0.71      0.71      0.71   1036038
weighted avg       0.71      0.71      0.71   1036038

              precision    recall  f1-score   support

           0       0.99      0.77      0.87    129504
           1       0.05      0.64      0.09      2478

    accuracy                           0.77    131982
   macro avg       0.52      0.71      0.48    131982
weighted avg       0.97      0.77      0.85    131982

False Positive Percentage:  0.9489077042399172  FP:  29363  TP:  1581
False Negative Rate:  0.008877847938399415  FN:  897  TN:  100141
{'10set': 0.5438995360119299, 'anc': 0.60541415226599, 'aog': 0.5529154390082265, 'arpa': 0.5827276669976558, 'asd': 0.5630436205142474, 'auto': 0.5172260432321362, 'babydoge': 0.5940445041340778, 'box': 0.9455

In [94]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier().fit(X_train, y_train)

y_train_pred = model1.predict(X_train)
y_test_pred = model1.predict(X_test)
# y_pred = model1.predict(X)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(y, y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

knn_pred = model1.predict(test_now)
knn_pred_proba = model1.predict_proba(test_now)
knn_pred_coin = {}
for i in range(len(knn_pred)):
    if knn_pred[i] == 1:
        knn_pred_coin[coin_list[i]] = knn_pred_proba[i][1]
print(knn_pred_coin)
print(len(knn_pred_coin))



              precision    recall  f1-score   support

           0       0.91      0.80      0.85     19848
           1       0.82      0.92      0.87     19848

    accuracy                           0.86     39696
   macro avg       0.87      0.86      0.86     39696
weighted avg       0.87      0.86      0.86     39696

              precision    recall  f1-score   support

           0       0.99      0.73      0.84    129524
           1       0.04      0.62      0.08      2458

    accuracy                           0.72    131982
   macro avg       0.52      0.67      0.46    131982
weighted avg       0.97      0.72      0.82    131982

False Positive Percentage:  0.9590307538880354  FP:  35581  TP:  1520
False Negative Rate:  0.009886067811258312  FN:  938  TN:  93943
{'ae': 0.6, 'alcx': 0.6, 'alu': 0.8, 'anc': 1.0, 'aog': 0.8, 'arpa': 0.8, 'ata': 0.8, 'atolo': 0.6, 'auction': 0.6, 'audio': 0.6, 'aurora': 0.8, 'ava': 1.0, 'babydoge': 0.6, 'badger': 0.8, 'bat': 0.8, 'bcd': 0.6

In [92]:
from sklearn.gaussian_process import GaussianProcessClassifier

model = GaussianProcessClassifier().fit(X_train, y_train)

y_train_pred = model.predict(X_train)
# y_test_pred = model.predict(X_test)
n_loop = int(len(X_test)/len(X_train))
y_test_pred = []
for i in range(n_loop+1):
    print((i+1)/n_loop)
    right_bound = np.min([(i+1)*len(X_train),len(prediktor_real)])
    X_partial = X_test.iloc[i*len(X_train):right_bound]
    y_target = X_partial['target']
    X_partial = X_partial[np.arange(2,17).tolist()]
    y_test_pred = y_test_pred + model.predict(X_partial).tolist()
print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))
# print(classification_report(prediktor_real['target'], y_pred))

tn, fp, fn, tp  = confusion_matrix(y_test, y_test_pred).ravel()

FPP = fp/(fp+tp)
FNP = fn/(tn+fn)

print('False Positive Percentage: ', FPP, ' FP: ', fp, ' TP: ', tp)
print('False Negative Rate: ', FNP, ' FN: ', fn, ' TN: ', tn)

gpr_pred = model.predict(test_now)
gpr_pred_proba = model.predict_proba(test_now)
gpr_pred_coin = {}
for i in range(len(gpr_pred)):
    if gpr_pred[i] == 1:
        gpr_pred_coin[coin_list[i]] = gpr_pred_proba[i][1]
print(gpr_pred_coin)
print(len(gpr_pred_coin))

MemoryError: Unable to allocate 2.91 GiB for an array with shape (19756, 19756) and data type float64

In [91]:
len(y_train)

19756

# Kotretan

In [30]:
len(prediktor_real.dropna())

699584

In [11]:
len(X_train)

1110906

In [19]:
df_checked_dict = {}
for i in df_checked.keys():
    df_checked_dict[i] = df_checked[i].to_dict()

In [21]:
import json

with open("df_checked_dict.json", "w") as outfile:
    json.dump(df_checked_dict, outfile)

In [6]:
import inspect
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

models = [RandomForestRegressor(), LinearRegression()]

for m in models:
    hyperparams = inspect.signature(m.__init__)
    print(hyperparams) # Do something with them here

(n_estimators=100, *, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
(*, fit_intercept=True, normalize=False, copy_X=True, n_jobs=None, positive=False)
